In [8]:
import cv2
import numpy as np
import cv2
import time
from IPython.display import clear_output

WIND_X = 480
WIND_Y = 360

def get_available_cameras (upper_bound = 10, lower_bound = 0):
    available = []
    
    for i in range (lower_bound, upper_bound):
        cap = cv2.VideoCapture (i)
    
        if (cap.isOpened ()):
            available.append (i)
    
        cap.release ()
    
    return available

def turn_to_RG (img):
    (h, w, d) = img.shape
    
    norm = np.zeros ((h, w), np.float)
    norm = img [:, :, 0].astype ('float') +\
           img [:, :, 1].astype ('float') +\
           img [:, :, 2].astype ('float')
    
    norm [norm == 0] = 5
    
    turned = np.zeros (img.shape, np.uint8)
    turned [:, :, 0] = ((img [:, :, 0].astype ('float')) / norm * 255).astype ('uint8')
    turned [:, :, 1] = ((img [:, :, 1].astype ('float')) / norm * 255).astype ('uint8')
    turned [:, :, 2] = ((img [:, :, 2].astype ('float')) / norm * 255).astype ('uint8')
    
    return turned

def obtain_color_ratio_mask (img, components, th, bl, use_rg = False):
    sh = img [:, :, 0].shape
    
    rg = img
    
    if (use_rg == True):
        rg = turn_to_RG (img)
    
    smoothed = cv2.blur (rg, (bl, bl))
    
    needed = img.copy ()
    needed [:, :, 0] = np.full (sh, components [0])
    needed [:, :, 1] = np.full (sh, components [1])
    needed [:, :, 2] = np.full (sh, components [2])
    
    diff = cv2.absdiff (smoothed, needed)
    
    dif = diff [:, :, 0] + diff [:, :, 1] + diff [:, :, 2]
    
    ret, res_mask = cv2.threshold (dif, th, 255, cv2.THRESH_BINARY_INV)
    
    res_mask = cv2.morphologyEx (res_mask, cv2.MORPH_ERODE, np.ones ((int (bl), int (bl)), np.uint8))
    res_mask = cv2.morphologyEx (res_mask, cv2.MORPH_CLOSE, np.ones ((int (bl), int (bl)), np.uint8))
    
    res = cv2.bitwise_and (img, img, mask = res_mask)
    
    return res, res_mask

def erase_little_parts (mask, area_th, hei_th, wid_th):
    result = np.array (mask)
    output = cv2.connectedComponentsWithStats (mask, 8, cv2.CV_32S)
    labels_count = output      [0]
    labels       = output      [1]
    stats        = output      [2]
    sz           = stats.shape [0]
    
    for label_num in range (0, sz - 1):
        if (stats [label_num, cv2.CC_STAT_AREA]   < area_th or
            stats [label_num, cv2.CC_STAT_WIDTH]  < wid_th  or
            stats [label_num, cv2.CC_STAT_HEIGHT] < hei_th):
            result [labels == label_num] = 0
    
    return result

def draw_lines (img, mask = None):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    kernel_size = 5
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)

    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny (blur_gray, low_threshold, high_threshold)

    rho             = 1           # distance resolution in pixels of the Hough grid
    theta           = np.pi / 180 # angular resolution in radians of the Hough grid
    threshold       = 45          # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 100         # minimum number of pixels making up a line
    max_line_gap    = 60          # maximum gap in pixels between connectable line segments
    
    line_image = np.copy (img) * 0  # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)
    
    if (lines is None):
        return img, np.zeros (img.shape, np.uint8)
    
    #print (len (lines))
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line (line_image, (x1, y1), (x2, y2), (255, 0, 0), 5)

            #cv2.circle (line_image, (x1, y1), 6, (20, 30, 240), -1)
            #cv2.circle (line_image, (x2, y2), 6, (20, 30, 240), -1)
        
    # Draw the lines on the  image
    lines_edges = cv2.addWeighted (img, 0.8, line_image, 1, 0)

    #cv2.imshow (name, lines_edges)
    
    return lines_edges, line_image

In [10]:
cam = cv2.VideoCapture (1)
#cam = cv2.VideoCapture ("inp.webm")

#cv2.namedWindow  ("field", cv2.WINDOW_NORMAL)
#cv2.resizeWindow ("field", (WIND_X, WIND_Y))

output_shape = (WIND_X * 3, WIND_Y * 2)

fourcc = cv2.VideoWriter_fourcc (*'XVID')
out    = cv2.VideoWriter        ('output.avi', fourcc, 20.0, output_shape)

cv2.namedWindow  ("lines detection", cv2.WINDOW_NORMAL)
cv2.resizeWindow ("lines detection", output_shape)

while (cam.isOpened ()):
    ret1, frame1 = cam.read ()
    
    if (ret1 != True):
        break
    
    frame_resized = cv2.resize (frame1, (int (WIND_X), int (WIND_Y)))
    
    cv2.waitKey (1)
    
    #field, thr_f = obtain_color_ratio_mask (frame_resized, (60, 180, 10), 170, 81, use_rg = True)
    
    field, f_mask = obtain_color_ratio_mask (frame_resized, (60, 180, 10), 170, 61, use_rg = True)
    
    #field, thr_f = obtain_color_ratio_mask (frame_resized, (60, 180, 10), 170, 41, use_rg = True)
    #field, thr_f = obtain_color_ratio_mask (frame_resized, (60, 180, 10), 170, 31, use_rg = True)
    #field, thr_f = obtain_color_ratio_mask (frame_resized, (60, 180, 10), 170, 21, use_rg = True)
    
    field2, f2_mask = obtain_color_ratio_mask (frame_resized, (60, 180, 10), 170, 5, use_rg = True)
    field_resized = cv2.resize (field, (WIND_X, WIND_Y))
    
    f2_mask_inv = cv2.bitwise_not (f2_mask)
    field_cut = cv2.bitwise_and (field, field, mask = f2_mask_inv)
    
    field_cut_resized = cv2.resize (field_cut, (WIND_X, WIND_Y))
    
    lines, lin = draw_lines (field_cut_resized)
    
    lines_mask = cv2.bitwise_and (f_mask, f2_mask_inv)
    lines_mask_resized = cv2.resize (lines_mask, (WIND_X, WIND_Y))
    lines_mask_resized_col = cv2.cvtColor (lines_mask_resized, cv2.COLOR_GRAY2RGB)
    
    #print (frame1.shape, lin.shape)
    lines_on_inp = cv2.addWeighted (frame_resized, 0.8, lin, 1, 0)
    
    row1 = np.concatenate ((frame_resized, field_resized, field_cut_resized), axis = 1)
    row2 = np.concatenate ((lines, lines_mask_resized_col, lines_on_inp), axis = 1)
    output_image = np.concatenate ((row1, row2), axis = 0)
    
    cv2.imshow ("lines detection", output_image)
    
    out.write (output_image)
    
    time.sleep  (0.01)
    
    if (cv2.waitKey (1) & 0xFF == ord ('q')):
        break

cam.release ()
out.release ()

cv2.waitKey           (1)
cv2.destroyAllWindows ()

In [19]:
frame1 = cv2.imread ("IMAGE1.jpg")

frame_resized = cv2.resize (frame1, (int (WIND_X), int (WIND_Y)))

cv2.waitKey (1)

#field, thr_f = obtain_color_ratio_mask (frame_resized, (60, 180, 10), 170, 81, use_rg = True)
field, f_mask = obtain_color_ratio_mask (frame_resized, (60, 180, 10), 170, 61, use_rg = True)
#field, thr_f = obtain_color_ratio_mask (frame_resized, (60, 180, 10), 170, 41, use_rg = True)
#field, thr_f = obtain_color_ratio_mask (frame_resized, (60, 180, 10), 170, 31, use_rg = True)
#field, thr_f = obtain_color_ratio_mask (frame_resized, (60, 180, 10), 170, 21, use_rg = True)

#field2, f2_mask = obtain_color_ratio_mask (frame_resized, (60, 180, 10), 170, 5, use_rg = True)
field2, f2_mask = obtain_color_ratio_mask (frame_resized, (60, 180, 10), 170, 1, use_rg = True)

field_resized = cv2.resize (field, (WIND_X, WIND_Y))

f2_mask_inv = cv2.bitwise_not (f2_mask)
field_cut = cv2.bitwise_and (field, field, mask = f2_mask_inv)

field_cut_resized = cv2.resize (field_cut, (WIND_X, WIND_Y))

lines, lin = draw_lines (field_cut_resized)

lines_mask = cv2.bitwise_and (f_mask, f2_mask_inv)
lines_mask_resized = cv2.resize (lines_mask, (WIND_X, WIND_Y))
lines_mask_resized_col = cv2.cvtColor (lines_mask_resized, cv2.COLOR_GRAY2RGB)

#print (frame1.shape, lin.shape)
lines_on_inp = cv2.addWeighted (frame_resized, 0.8, lin, 1, 0)

row1 = np.concatenate ((frame_resized, field_resized, field_cut_resized), axis = 1)
row2 = np.concatenate ((lines, lines_mask_resized_col, lines_on_inp), axis = 1)
output_image = np.concatenate ((row1, row2), axis = 0)

cv2.imshow ("lines detection", output_image)

cv2.waitKey           (0)
cv2.destroyAllWindows ()